## FIT3182 ASSIGNMENT 2 Part A
#####  Student ID: 32203004
#####  Student Email: cgoh0016@student.monash.edu

### Task 1:  MongoDB Data Model

In [ ]:
{
    "_id": ObjectId("..."),  
    "station": 948700,
    "date": ISODate("2023-11-21"),
    "hotspots": [{
                    "time": item.isoformat(),
                    "coordinates": [lat, long],
                    "confidence": int(row["confidence"]),  
                    "surface_temperature_celcius": float(row["surface_temperature_celcius"])
                    }]
    "ghi": 250
    "air_temperature_celcius": 22.1,
    "relative_humidity": 65,
    "windspeed_knots": 10,
    "max_wind_speed": 15,
    "precipitation": 0.0,
  
}

### Justification

First off, here are the findings from analyzing the queries and the dataset:
1. One climate record may be linked with multiple hotspot records resulting in a one to many relationship
2. Data Redundancy (Potential): Embedding hotspot data within the climate records could lead to some data redundancy, especially as the collection size grows. However, for the dataset size likely encountered in Task 2, this redundancy is manageable, and the benefits outweigh the drawbacks.

Hence, by combining the data via embedding, you can directly analyze how factors like air temperature, humidity, or wind speed relate to the surface temperature hotspots. This would be much more difficult if you had to query and join the data from separate collections every time.

In addition, if you frequently need to retrieve both hotspot data and relevant climate data for a specific date and location (in this case in **Task 2: Question 2C, 2D** and so on, embedding them into one collection simplifies queries. You can write a single query to get the information you need, rather than needing to query and join data from separate collections.

Moreover, when the data is embedded, you can create visualizations that combine hotspot locations with climate data. This can provide a more comprehensive understanding of the relationship between the two. For example, you could create a map that shows hotspots overlaid with contours of air temperature.

This may however, result in data redundancy, especially as the collection size grows. This would not be a large concern or affect the database as it would be impossible for the amount of fires to exceed the maximum file size per document for MongoDB.

In addition, we can easily rectify this by introducing denormalization and indexing.

###  Task 2. Querying MongoDB using PyMongo (15%)
#### Task 2.1


In [7]:
# IMPORTING 
import numpy as np
import pandas as pd 
from pymongo import MongoClient

In [8]:
# READING CSV FOR CHECKING PURPOSES
climate = pd.read_csv('/home/student/A2/Dataset/climate_historic.csv')
hotspot = pd.read_csv('/home/student/A2/Dataset/hotspot_historic.csv')

In [9]:
# CONNECTING TO MONGO
client = MongoClient('10.192.104.132', 27017)
db = client["fit3182_assignment_db"]

climate_data_collection = db["climate_data"]

In [4]:
climate_data_collection.drop()


In [5]:
import csv
import datetime

def insert_document(collection, data):
  try:
    collection.insert_one(data)
  except Exception as e:
    print(f"Error inserting document: {e}")

# Read hotspot data
with open(r"/home/student/A2/Dataset/climate_historic.csv") as csvfile:
  reader = csv.DictReader(csvfile)
  for row in reader:
    item = datetime.datetime.strptime(row["date"], "%d/%m/%Y").date().isoformat()
    
    with open("/home/student/A2/Dataset/hotspot_historic.csv") as hotspot_file:
            hotspot_reader = csv.DictReader(hotspot_file)
            matching_hotspots = []
            for hotspot_row in hotspot_reader:
                hotspot_date = datetime.datetime.strptime(hotspot_row["date"], "%d/%m/%Y").date().isoformat()
                if hotspot_date == item:
                    matching_hotspots.append({
                        "time": datetime.datetime.strptime(hotspot_row["datetime"], "%Y-%m-%d %H:%M:%S").time().isoformat(),
                        "latitude": float(hotspot_row["latitude"]),
                        "longitude": float(hotspot_row["longitude"]),
                        "surface_temperature_celcius": float(hotspot_row["surface_temperature_celcius"]),
                        "confidence": float(hotspot_row["confidence"])
                    })
    
    data = {
            "date": item,
            "station": row["station"],
            "air_temperature_celcius": float(row["air_temperature_celcius"]),
            "relative_humidity": float(row["relative_humidity"]),
            "windspeed_knots": float(row["windspeed_knots"]),
            "max_wind_speed": float(row["max_wind_speed"]),
            "precipitation": (row["precipitation"]),  
            "GHI_w/m2": float(row["GHI_w/m2"]),
            "hotspots": matching_hotspots
        }
    insert_document(climate_data_collection, data)



In [11]:
# testing
from pprint import pprint
pprint(client.list_database_names())
pprint(db.list_collection_names())

climatecount = climate_data_collection.count_documents({})  
# hotspotcount = hotspots_collection.count_documents({}) 

climatecsvcount = climate.shape[0]
hotspotcsvcount = hotspot.shape[0]

print(f"Number of documents in climate data: {climatecount}")
# print(f"Number of documents in hotspot data: {hotspotcount}")
print(f"Number of rows in climate data CSV: {climatecsvcount}")
print(f"Number of rows in hotspot data CSV: {hotspotcsvcount}")

['admin', 'config', 'fit3182_assignment_db', 'fit3182_db', 'local']
['climate_data']
Number of documents in climate data: 366
Number of rows in climate data CSV: 366
Number of rows in hotspot data CSV: 2668


In [12]:
document1 = climate_data_collection.find_one()
pprint(document1)

random_records = climate_data_collection.aggregate([{ "$sample": { "size": 5 } }])
for record in random_records:
    pprint(record)

{'GHI_w/m2': 154.0,
 '_id': ObjectId('6654141b4f0eb9e1d0be3797'),
 'air_temperature_celcius': 19.0,
 'date': '2022-12-31',
 'hotspots': [],
 'max_wind_speed': 11.1,
 'precipitation': '0.00I',
 'relative_humidity': 56.8,
 'station': '948700',
 'windspeed_knots': 7.9}
{'GHI_w/m2': 183.0,
 '_id': ObjectId('6654142c4f0eb9e1d0be38de'),
 'air_temperature_celcius': 23.0,
 'date': '2023-11-24',
 'hotspots': [],
 'max_wind_speed': 15.0,
 'precipitation': '0.01G',
 'relative_humidity': 58.8,
 'station': '948702',
 'windspeed_knots': 9.1}
{'GHI_w/m2': 82.0,
 '_id': ObjectId('665414264f0eb9e1d0be3853'),
 'air_temperature_celcius': 9.0,
 'date': '2023-07-08',
 'hotspots': [],
 'max_wind_speed': 13.0,
 'precipitation': '0.08G',
 'relative_humidity': 42.8,
 'station': '948701',
 'windspeed_knots': 9.3}
{'GHI_w/m2': 82.0,
 '_id': ObjectId('665414274f0eb9e1d0be3871'),
 'air_temperature_celcius': 9.0,
 'date': '2023-08-07',
 'hotspots': [],
 'max_wind_speed': 15.9,
 'precipitation': '0.12G',
 'relative_

#### Task 2.2 MongoDB Queries

In [13]:
# QUESTION A
a = list(climate_data_collection.find({'date': '2023-12-12'}))
print("Climate data on 12th December 2023:")
for record in a:
    pprint(record)

Climate data on 12th December 2023:
{'GHI_w/m2': 156.0,
 '_id': ObjectId('6654142d4f0eb9e1d0be38f0'),
 'air_temperature_celcius': 19.0,
 'date': '2023-12-12',
 'hotspots': [{'confidence': 53.0,
               'latitude': -37.903,
               'longitude': 145.25,
               'surface_temperature_celcius': 44.0,
               'time': '00:45:38'}],
 'max_wind_speed': 12.0,
 'precipitation': '0.00I',
 'relative_humidity': 55.3,
 'station': '948702',
 'windspeed_knots': 6.2}


In [14]:
# QUESTION B
# Find the latitude, longitude, surface temperature (°C), and confidence 
# when the surface temperature (°C) was between 65 °C and 100 °C
b = list(climate_data_collection.aggregate([{'$unwind': '$hotspots'}, 
                                                 {'$match': {'hotspots.surface_temperature_celcius': {'$gte': 65, '$lte': 100}}}, 
                                                 {'$project': {'_id': 0, 'latitude': '$hotspots.latitude', 
                               'longitude': '$hotspots.latitude', 
                               'surface_temperature_celcius': '$hotspots.surface_temperature_celcius'}}]))

print(len(b))
print("\nHotspot data when surface temperature was between 65 °C and 100 °C:")
for record in b:
    pprint(record)

476

Hotspot data when surface temperature was between 65 °C and 100 °C:
{'latitude': -37.2284,
 'longitude': -37.2284,
 'surface_temperature_celcius': 73.0}
{'latitude': -37.6572,
 'longitude': -37.6572,
 'surface_temperature_celcius': 80.0}
{'latitude': -37.0193,
 'longitude': -37.0193,
 'surface_temperature_celcius': 71.0}
{'latitude': -37.4229,
 'longitude': -37.4229,
 'surface_temperature_celcius': 99.0}
{'latitude': -37.0055,
 'longitude': -37.0055,
 'surface_temperature_celcius': 68.0}
{'latitude': -37.4128,
 'longitude': -37.4128,
 'surface_temperature_celcius': 98.0}
{'latitude': -34.357, 'longitude': -34.357, 'surface_temperature_celcius': 67.0}
{'latitude': -34.3539,
 'longitude': -34.3539,
 'surface_temperature_celcius': 72.0}
{'latitude': -36.9939,
 'longitude': -36.9939,
 'surface_temperature_celcius': 68.0}
{'latitude': -36.9959,
 'longitude': -36.9959,
 'surface_temperature_celcius': 75.0}
{'latitude': -34.3654,
 'longitude': -34.3654,
 'surface_temperature_celcius': 65

In [15]:
# QUESTION C
# Find date, surface temperature (°C), air temperature (°C), relative humidity and maxwind speed 
# on 15th and 16th of December 2023.
c = list(climate_data_collection.aggregate([
    {'$match': {'date': {'$in': ['2023-12-15', '2023-12-16']}}},
    {'$unwind': '$hotspots'},
    {'$project': {
        '_id': 0,
        'date': 1,
        'surface_temperature_celcius': '$hotspots.surface_temperature_celcius',
        'air_temperature_celcius': 1,
        'relative_humidity': 1,
        'max_wind_speed': 1
    }}
]))

print(len(c))

print("\nData on 15th and 16th December 2023:")
for record in c:
    pprint(record)


19

Data on 15th and 16th December 2023:
{'air_temperature_celcius': 18.0,
 'date': '2023-12-15',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celcius': 42.0}
{'air_temperature_celcius': 18.0,
 'date': '2023-12-15',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celcius': 36.0}
{'air_temperature_celcius': 18.0,
 'date': '2023-12-15',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celcius': 38.0}
{'air_temperature_celcius': 18.0,
 'date': '2023-12-15',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celcius': 40.0}
{'air_temperature_celcius': 18.0,
 'date': '2023-12-16',
 'max_wind_speed': 13.0,
 'relative_humidity': 53.7,
 'surface_temperature_celcius': 43.0}
{'air_temperature_celcius': 18.0,
 'date': '2023-12-16',
 'max_wind_speed': 13.0,
 'relative_humidity': 53.7,
 'surface_temperature_celcius': 33.0}
{'air_temperature_celcius': 18.0,
 'date': '2023-12-16',
 'max_wind_spe

In [16]:
# QUESTION D:  Find datetime, air temperature (°C), surface temperature (°C) and confidence
# when the confidence is between 80 and 100

d = list(climate_data_collection.aggregate([
    {'$unwind': '$hotspots'},
    {'$match': {'hotspots.confidence': {'$gte': 80, '$lte': 100}}}, 
    {'$project': {
    '_id': 0,
    'date': 1,
    'hotspots.time': 1,
    'air_temperature_celcius': 1,
    'hotspots.surface_temperature_celcius': 1,
    'hotspots.confidence': 1
    }}]))
     

print(len(d))
print("\nWhen confidence is between 80 and 100:")
for record in d:
    pprint(record)

1133

When confidence is between 80 and 100:
{'air_temperature_celcius': 20.0,
 'date': '2023-03-06',
 'hotspots': {'confidence': 87.0,
              'surface_temperature_celcius': 62.0,
              'time': '05:06:30'}}
{'air_temperature_celcius': 20.0,
 'date': '2023-03-06',
 'hotspots': {'confidence': 85.0,
              'surface_temperature_celcius': 59.0,
              'time': '05:06:20'}}
{'air_temperature_celcius': 19.0,
 'date': '2023-03-07',
 'hotspots': {'confidence': 88.0,
              'surface_temperature_celcius': 64.0,
              'time': '04:16:10'}}
{'air_temperature_celcius': 23.0,
 'date': '2023-03-09',
 'hotspots': {'confidence': 86.0,
              'surface_temperature_celcius': 41.0,
              'time': '13:23:40'}}
{'air_temperature_celcius': 19.0,
 'date': '2023-03-10',
 'hotspots': {'confidence': 100.0,
              'surface_temperature_celcius': 105.0,
              'time': '04:48:40'}}
{'air_temperature_celcius': 19.0,
 'date': '2023-03-10',
 'hotspots'

 'hotspots': {'confidence': 85.0,
              'surface_temperature_celcius': 59.0,
              'time': '04:22:10'}}
{'air_temperature_celcius': 19.0,
 'date': '2023-04-06',
 'hotspots': {'confidence': 90.0,
              'surface_temperature_celcius': 67.0,
              'time': '04:22:10'}}
{'air_temperature_celcius': 19.0,
 'date': '2023-04-06',
 'hotspots': {'confidence': 90.0,
              'surface_temperature_celcius': 66.0,
              'time': '04:21:30'}}
{'air_temperature_celcius': 19.0,
 'date': '2023-04-06',
 'hotspots': {'confidence': 94.0,
              'surface_temperature_celcius': 96.0,
              'time': '04:21:10'}}
{'air_temperature_celcius': 19.0,
 'date': '2023-04-06',
 'hotspots': {'confidence': 100.0,
              'surface_temperature_celcius': 115.0,
              'time': '04:21:00'}}
{'air_temperature_celcius': 19.0,
 'date': '2023-04-06',
 'hotspots': {'confidence': 84.0,
              'surface_temperature_celcius': 58.0,
              'time': '04:21

{'air_temperature_celcius': 15.0,
 'date': '2023-04-18',
 'hotspots': {'confidence': 93.0,
              'surface_temperature_celcius': 73.0,
              'time': '04:44:50'}}
{'air_temperature_celcius': 15.0,
 'date': '2023-04-18',
 'hotspots': {'confidence': 93.0,
              'surface_temperature_celcius': 72.0,
              'time': '04:44:50'}}
{'air_temperature_celcius': 15.0,
 'date': '2023-04-18',
 'hotspots': {'confidence': 80.0,
              'surface_temperature_celcius': 53.0,
              'time': '04:44:50'}}
{'air_temperature_celcius': 15.0,
 'date': '2023-04-18',
 'hotspots': {'confidence': 94.0,
              'surface_temperature_celcius': 75.0,
              'time': '04:44:50'}}
{'air_temperature_celcius': 15.0,
 'date': '2023-04-18',
 'hotspots': {'confidence': 84.0,
              'surface_temperature_celcius': 58.0,
              'time': '04:44:50'}}
{'air_temperature_celcius': 15.0,
 'date': '2023-04-18',
 'hotspots': {'confidence': 89.0,
              'surface_t

In [17]:
# QUESTION E:
# Find the top 10 records with the highest surface temperature (°C)

e = list(climate_data_collection.aggregate([{'$unwind': '$hotspots'}, 
                                            {'$sort' : {'hotspots.surface_temperature_celcius': -1}},
                                            {'$limit': 10}]))
# print(len(e))
print("Top 10 records with the highest surface temperature (°C):")
for record in e:
    pprint(record)

Top 10 records with the highest surface temperature (°C):
{'GHI_w/m2': 122.0,
 '_id': ObjectId('665414224f0eb9e1d0be3802'),
 'air_temperature_celcius': 15.0,
 'date': '2023-04-18',
 'hotspots': {'confidence': 100.0,
              'latitude': -38.1665,
              'longitude': 143.062,
              'surface_temperature_celcius': 124.0,
              'time': '04:52:00'},
 'max_wind_speed': 9.9,
 'precipitation': '0.00I',
 'relative_humidity': 56.1,
 'station': '948701',
 'windspeed_knots': 5.1}
{'GHI_w/m2': 140.0,
 '_id': ObjectId('665414214f0eb9e1d0be37f4'),
 'air_temperature_celcius': 16.0,
 'date': '2023-04-04',
 'hotspots': {'confidence': 100.0,
              'latitude': -36.343,
              'longitude': 142.1986,
              'surface_temperature_celcius': 123.0,
              'time': '04:32:50'},
 'max_wind_speed': 12.0,
 'precipitation': '0.00I',
 'relative_humidity': 47.5,
 'station': '948701',
 'windspeed_knots': 5.4}
{'GHI_w/m2': 121.0,
 '_id': ObjectId('665414224f0eb9e1d

In [18]:
# QUESTION F: Find the number of fires each day. 
# Display the total number of fires and the date in the output.

# no of fires is the no of incidents per date in the hotspot collection
f = list(climate_data_collection.aggregate( [{'$project': {'_id': 0, 'date': 1, 
                                                           'total_fires': {'$size': {'$ifNull': ['$hotspots', []]}}}}]))
print("Number of fires each day:")
for record in f:
    print(f"Date: {record['date']}, Total fires: {record['total_fires']}")


Number of fires each day:
Date: 2022-12-31, Total fires: 0
Date: 2023-01-02, Total fires: 0
Date: 2023-01-03, Total fires: 0
Date: 2023-01-04, Total fires: 0
Date: 2023-01-05, Total fires: 0
Date: 2023-01-06, Total fires: 0
Date: 2023-01-07, Total fires: 0
Date: 2023-01-08, Total fires: 0
Date: 2023-01-09, Total fires: 0
Date: 2023-01-10, Total fires: 0
Date: 2023-01-11, Total fires: 0
Date: 2023-01-12, Total fires: 0
Date: 2023-01-13, Total fires: 0
Date: 2023-01-14, Total fires: 0
Date: 2023-01-15, Total fires: 0
Date: 2023-01-16, Total fires: 0
Date: 2023-01-17, Total fires: 0
Date: 2023-01-18, Total fires: 0
Date: 2023-01-19, Total fires: 0
Date: 2023-01-20, Total fires: 0
Date: 2023-01-21, Total fires: 0
Date: 2023-01-22, Total fires: 0
Date: 2023-01-23, Total fires: 0
Date: 2023-01-24, Total fires: 0
Date: 2023-01-25, Total fires: 0
Date: 2023-01-26, Total fires: 0
Date: 2023-01-27, Total fires: 0
Date: 2023-01-28, Total fires: 0
Date: 2023-01-29, Total fires: 0
Date: 2023-01-30,

In [19]:
# QUESTION G: Find the records of fires where the confidence is below 70

g = list(climate_data_collection.aggregate([
    {'$unwind': '$hotspots'}, 
    {'$match': {'hotspots.confidence': {'$lt': 70}}}, 
    {'$project': {'_id': 0, 'hotspots': 1}}]))
print(len(g))
print("Records of fires where the confidence is below 7:")
for record in g:
    pprint(record)

782
Records of fires where the confidence is below 7:
{'hotspots': {'confidence': 68.0,
              'latitude': -37.7885,
              'longitude': 141.9352,
              'surface_temperature_celcius': 55.0,
              'time': '04:51:00'}}
{'hotspots': {'confidence': 54.0,
              'latitude': -37.7171,
              'longitude': 147.5866,
              'surface_temperature_celcius': 44.0,
              'time': '03:57:00'}}
{'hotspots': {'confidence': 55.0,
              'latitude': -36.2544,
              'longitude': 148.0353,
              'surface_temperature_celcius': 42.0,
              'time': '04:43:00'}}
{'hotspots': {'confidence': 54.0,
              'latitude': -37.2197,
              'longitude': 147.9621,
              'surface_temperature_celcius': 43.0,
              'time': '04:42:30'}}
{'hotspots': {'confidence': 56.0,
              'latitude': -37.0286,
              'longitude': 148.155,
              'surface_temperature_celcius': 42.0,
              'ti

{'hotspots': {'confidence': 61.0,
              'latitude': -36.9908,
              'longitude': 141.2448,
              'surface_temperature_celcius': 41.0,
              'time': '04:45:40'}}
{'hotspots': {'confidence': 63.0,
              'latitude': -36.4428,
              'longitude': 145.5639,
              'surface_temperature_celcius': 43.0,
              'time': '04:45:40'}}
{'hotspots': {'confidence': 67.0,
              'latitude': -38.1159,
              'longitude': 143.7887,
              'surface_temperature_celcius': 61.0,
              'time': '04:45:40'}}
{'hotspots': {'confidence': 54.0,
              'latitude': -37.8084,
              'longitude': 143.1985,
              'surface_temperature_celcius': 46.0,
              'time': '04:45:40'}}
{'hotspots': {'confidence': 56.0,
              'latitude': -36.4147,
              'longitude': 141.0669,
              'surface_temperature_celcius': 39.0,
              'time': '04:45:10'}}
{'hotspots': {'confidence': 69.0,
 

In [20]:
# QUESTION H: Find the average surface temperature (°C) for each day. 
# Display average surface temperature (°C) and the date in the output

h = list(climate_data_collection.aggregate(
    [{'$unwind': '$hotspots'},
     {'$group': {'_id': '$date', 
                 'avg_surface_temp': {
                     '$avg': '$hotspots.surface_temperature_celcius'}}}]))
print("Average surface temperature (°C) for each day:")
for record in h:
    pprint(f"Date: {record['_id']}, Average Surface Temperature (°C): {record['avg_surface_temp']}")

Average surface temperature (°C) for each day:
'Date: 2023-10-07, Average Surface Temperature (°C): 42.0'
'Date: 2023-12-09, Average Surface Temperature (°C): 58.25'
'Date: 2023-03-06, Average Surface Temperature (°C): 60.5'
'Date: 2023-10-15, Average Surface Temperature (°C): 72.66666666666667'
'Date: 2023-12-27, Average Surface Temperature (°C): 62.75'
'Date: 2023-04-18, Average Surface Temperature (°C): 53.36615384615384'
'Date: 2023-09-27, Average Surface Temperature (°C): 49.714285714285715'
'Date: 2023-05-07, Average Surface Temperature (°C): 50.333333333333336'
'Date: 2023-05-23, Average Surface Temperature (°C): 51.2'
'Date: 2023-03-09, Average Surface Temperature (°C): 46.666666666666664'
'Date: 2023-03-29, Average Surface Temperature (°C): 51.0'
'Date: 2023-04-08, Average Surface Temperature (°C): 60.75'
'Date: 2023-04-16, Average Surface Temperature (°C): 48.72222222222222'
'Date: 2023-06-07, Average Surface Temperature (°C): 51.857142857142854'
'Date: 2023-11-08, Average Su

In [21]:
# QUESTION I: Find the top 10 records with the lowest GHI.
i = list(climate_data_collection.aggregate([{'$sort': {'ghi': 1}},{'$limit': 10}]))
print("Top 10 records with the lowest GHI:")
for record in i:
    pprint(record)

Top 10 records with the lowest GHI:
{'GHI_w/m2': 154.0,
 '_id': ObjectId('6654141c4f0eb9e1d0be379f'),
 'air_temperature_celcius': 19.0,
 'date': '2023-01-09',
 'hotspots': [],
 'max_wind_speed': 8.9,
 'precipitation': '0.00I',
 'relative_humidity': 56.3,
 'station': '948700',
 'windspeed_knots': 5.8}
{'GHI_w/m2': 161.0,
 '_id': ObjectId('6654141c4f0eb9e1d0be37a0'),
 'air_temperature_celcius': 20.0,
 'date': '2023-01-10',
 'hotspots': [],
 'max_wind_speed': 13.0,
 'precipitation': '0.00I',
 'relative_humidity': 57.0,
 'station': '948700',
 'windspeed_knots': 8.7}
{'GHI_w/m2': 185.0,
 '_id': ObjectId('6654141b4f0eb9e1d0be379b'),
 'air_temperature_celcius': 24.0,
 'date': '2023-01-05',
 'hotspots': [],
 'max_wind_speed': 13.0,
 'precipitation': '0.00I',
 'relative_humidity': 62.3,
 'station': '948700',
 'windspeed_knots': 7.0}
{'GHI_w/m2': 160.0,
 '_id': ObjectId('6654141c4f0eb9e1d0be379e'),
 'air_temperature_celcius': 20.0,
 'date': '2023-01-08',
 'hotspots': [],
 'max_wind_speed': 18.1,

In [22]:
# QUESTION J: Find the records with a 24-hour precipitation recorded between 0.20 to 0.35.
# we want it ending with G, G=1reportof24-hourprecipitation amount.

j = list(climate_data_collection.aggregate(
    [{'$match': {'precipitation': {'$gt': '0.20G','$lt': '0.35G'}}
    }]))
print(len(j))
print("Records with a 24-hour precipitation recorded between 0.20 to 0.35:")
for record in j:
    pprint(record)

13
Records with a 24-hour precipitation recorded between 0.20 to 0.35:
{'GHI_w/m2': 157.0,
 '_id': ObjectId('6654141c4f0eb9e1d0be37a3'),
 'air_temperature_celcius': 19.0,
 'date': '2023-01-13',
 'hotspots': [],
 'max_wind_speed': 18.1,
 'precipitation': '0.31G',
 'relative_humidity': 54.1,
 'station': '948700',
 'windspeed_knots': 11.2}
{'GHI_w/m2': 146.0,
 '_id': ObjectId('665414214f0eb9e1d0be37ee'),
 'air_temperature_celcius': 17.0,
 'date': '2023-03-29',
 'hotspots': [{'confidence': 69.0,
               'latitude': -34.2648,
               'longitude': 141.6325,
               'surface_temperature_celcius': 51.0,
               'time': '00:48:40'}],
 'max_wind_speed': 21.0,
 'precipitation': '0.24G',
 'relative_humidity': 49.9,
 'station': '948701',
 'windspeed_knots': 12.2}
{'GHI_w/m2': 166.0,
 '_id': ObjectId('665414224f0eb9e1d0be3804'),
 'air_temperature_celcius': 20.0,
 'date': '2023-04-20',
 'hotspots': [{'confidence': 84.0,
               'latitude': -36.8871,
               '

### Task 2.3 Optimization

In [23]:
# Get the admin database
admin_db = client.admin

# Run the currentOp command
current_op_result = admin_db.command({"currentOp": 1})

# Print the result
pprint(current_op_result)

{'inprog': [{'active': True,
             'client': '172.17.0.1:43978',
             'clientMetadata': {'driver': {'name': 'PyMongo',
                                           'version': '4.3.3'},
                                'os': {'architecture': 'x86_64',
                                       'name': 'Linux',
                                       'type': 'Linux',
                                       'version': '5.15.133.1-microsoft-standard-WSL2'},
                                'platform': 'CPython 3.8.16.final.0'},
             'command': {'$db': 'admin',
                         'currentOp': 1,
                         'lsid': {'id': Binary(b'\xbe\t\x19P\x19\x89G\x0c\x9cpJ\xa4\xe2`[D', 4)}},
             'connectionId': 6,
             'currentOpTime': '2024-05-27T15:06:18.771+10:00',
             'desc': 'conn6',
             'flowControlStats': {},
             'host': '77a8ae913ea1:27017',
             'lockStats': {},
             'locks': {},
             'lsid': {'

Overall, this performance snapshot indicates that there are multiple active operations, mostly command-related, within the MongoDB deployment. The operations seem to be initiated by clients using PyMongo drivers, running on both Windows and Linux platforms. They are generally performing well, with varying durations and some minor waits observed.

In [24]:
query = {'precipitation': {'$gte': '0.20G','$lte': '0.35G'}}

# Use $explain to evaluate the query
query_explain = climate_data_collection.find(query).explain()

# Print the explanation
pprint(query_explain)

{'command': {'$db': 'fit3182_assignment_db',
             'filter': {'precipitation': {'$gte': '0.20G', '$lte': '0.35G'}},
             'find': 'climate_data'},
 'executionStats': {'allPlansExecution': [],
                    'executionStages': {'advanced': 19,
                                        'direction': 'forward',
                                        'docsExamined': 366,
                                        'executionTimeMillisEstimate': 0,
                                        'filter': {'$and': [{'precipitation': {'$lte': '0.35G'}},
                                                            {'precipitation': {'$gte': '0.20G'}}]},
                                        'isEOF': 1,
                                        'nReturned': 19,
                                        'needTime': 348,
                                        'needYield': 0,
                                        'restoreState': 0,
                                        'saveState': 0,
    

In [25]:
# happens 5 times
hotspots_collection.create_index([('surface_temperature_celcius', 1)])

#happens 3 times
hotspots_collection.create_index([('confidence', 1)])

# happens 2 times
hotspots_collection.create_index([('air_temperature_celcius', 1)])
hotspots_collection.create_index([('date', 1), ('surface_temperature_celcius', 1)])
hotspots_collection.create_index([('date', -1)])
hotspots_collection.create_index([('GHI_w/m2', 1)])
hotspots_collection.create_index([('precipitation', 1)])


NameError: name 'hotspots_collection' is not defined

For the queries mentioned in Task 2, typical fields involved include date, surface_temperature_celcius, confidence, GHI_w/m2, precipitation, etc.

Justification for Index Design:

Indexes on date, surface_temperature_celcius, confidence, GHI_w/m2, and precipitation are essential for the given queries.
Single-field indexes are efficient for queries filtering on a single field. The chosen index design for date, surface_temperature_celcius, confidence, GHI_w/m2, and precipitation strikes a balance between efficiency and resource utilization. Single-field indexes cater to individual field queries, while potential compound indexes can be explored based on the specific query patterns in Task 2. By monitoring query performance and analyzing data distribution, further optimizations to the index strategy can be made.